Провести сравнение RNN, LSTM, GRU на датасете отзывов (из предыдущих занятий/материалов)

In [2]:
import pandas as pd
import numpy as np
import tensorflow as tf
import emoji
from nltk import word_tokenize
from string import punctuation, ascii_letters
from stop_words import get_stop_words
from pymorphy2 import MorphAnalyzer
from collections import Counter
from sklearn.model_selection import train_test_split
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras import regularizers
from gensim.models import Word2Vec

In [3]:
MAX_WORDS = 5000
MAX_LEN = 20
EMB_SIZE = 100

In [9]:
morph = MorphAnalyzer()

In [10]:
stop_words = set(get_stop_words('ru'))

In [11]:
punctuation = set(punctuation).union((' ', '«', '»', '—', '–', '“', '”', '…'))

In [12]:
cyrillic_letters = set([chr(i) for i in range(ord('а'), ord('я') + 1)] +
                       [chr(i) for i in range(ord('А'), ord('Я') + 1)] +
                       ['ё', 'Ё'])

In [13]:
data = pd.read_excel("отзывы за лето.xls")
data.columns = ['rating', 'content', 'date']

In [14]:
data['target'] = (data.rating > 3).astype(int)

In [15]:
def preprocess(text):
    tokens = word_tokenize(text)
    result = []
    for token in tokens:
        if (set(token).intersection(cyrillic_letters)
            or set(token).intersection(set(ascii_letters))
            or token in emoji.UNICODE_EMOJI):
            result.append(token)
    tokens = [token.lower() for token in result if token.lower() not in stop_words]
    tokens = [token for token in tokens if token in emoji.UNICODE_EMOJI or len(token) >  1]
    tokens = [morph.parse(token)[0].normal_form for token in tokens]
    return tokens

In [16]:
data['processed'] = data.content.apply(lambda x: " ".join(preprocess(str(x))))

In [17]:
X_train, X_val, y_train, y_val = train_test_split(data['processed'], data['target'], test_size=0.2,
                                                    random_state=42, stratify=data['target'])

In [18]:
tokenizer = tf.keras.preprocessing.text.Tokenizer(num_words=MAX_WORDS, oov_token='UNK')

In [19]:
tokenizer.fit_on_texts(X_train)

In [20]:
X_train = pad_sequences(tokenizer.texts_to_sequences(X_train),
                        maxlen=MAX_LEN,
                        padding='post',
                        truncating='post')

In [21]:
X_val = pad_sequences(tokenizer.texts_to_sequences(X_val),
                        maxlen=MAX_LEN,
                        padding='post',
                        truncating='post')

In [22]:
vocab = tokenizer.index_word

In [23]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
rnn = tf.keras.layers.SimpleRNN(128, recurrent_dropout=0.2, activation='relu')(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(rnn)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [24]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [25]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [26]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [27]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 16527 samples, validate on 4132 samples
Epoch 1/10
16527/16527 [==============================] - 6s 365us/sample - loss: 0.4576 - accuracy: 0.8014 - val_loss: 0.4074 - val_accuracy: 0.8427
Epoch 2/10
16527/16527 [==============================] - 2s 149us/sample - loss: 0.2876 - accuracy: 0.8736 - val_loss: 0.2543 - val_accuracy: 0.9025
Epoch 3/10
16527/16527 [==============================] - 2s 140us/sample - loss: 0.2452 - accuracy: 0.9055 - val_loss: 0.3381 - val_accuracy: 0.8974
Epoch 4/10
16527/16527 [==============================] - 2s 151us/sample - loss: 0.2222 - accuracy: 0.9163 - val_loss: 0.2868 - val_accuracy: 0.8998
Epoch 5/10
16527/16527 [==============================] - 2s 142us/sample - loss: 0.1980 - accuracy: 0.9265 - val_loss: 0.3148 - val_accuracy: 0.8969
Epoch 6/10
16527/16527 [==============================] - 2s 147us/sample - loss: 0.2005 - accuracy: 0.9150 - val_loss: 0.3403 - val_accuracy: 0.7713
Epoch 7/10
16527/16527 [===========================

In [28]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
lstm = tf.keras.layers.Bidirectional(tf.keras.layers.LSTM(64))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(lstm)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [29]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [30]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [31]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [32]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 16527 samples, validate on 4132 samples
Epoch 1/10
16527/16527 [==============================] - 8s 468us/sample - loss: 0.3769 - accuracy: 0.8434 - val_loss: 0.2301 - val_accuracy: 0.8988
Epoch 2/10
16527/16527 [==============================] - 5s 282us/sample - loss: 0.1981 - accuracy: 0.9160 - val_loss: 0.2061 - val_accuracy: 0.9126
Epoch 3/10
16527/16527 [==============================] - 4s 258us/sample - loss: 0.1650 - accuracy: 0.9352 - val_loss: 0.2082 - val_accuracy: 0.9131
Epoch 4/10
16527/16527 [==============================] - 4s 264us/sample - loss: 0.1461 - accuracy: 0.9451 - val_loss: 0.2265 - val_accuracy: 0.9102
Epoch 5/10
16527/16527 [==============================] - 4s 254us/sample - loss: 0.1295 - accuracy: 0.9523 - val_loss: 0.2398 - val_accuracy: 0.9083
Epoch 6/10
16527/16527 [==============================] - 4s 258us/sample - loss: 0.1176 - accuracy: 0.9578 - val_loss: 0.2935 - val_accuracy: 0.8986
Epoch 7/10
16527/16527 [===========================

In [33]:
inputs = tf.keras.layers.Input(shape=(MAX_LEN,))
embeddings = tf.keras.layers.Embedding(input_dim=len(vocab),
                                       output_dim=EMB_SIZE,
                                       activity_regularizer=regularizers.l2(1e-6))(inputs)
gru = tf.keras.layers.Bidirectional(tf.keras.layers.GRU(64))(embeddings)
dense = tf.keras.layers.Dense(64, activation='relu')(gru)
outputs = tf.keras.layers.Dense(1, activation='sigmoid')(dense)

In [34]:
model = tf.keras.Model(inputs=inputs, outputs=outputs)

In [35]:
optimizer = tf.keras.optimizers.Adam(learning_rate=0.001)

In [36]:
model.compile(optimizer=optimizer,
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [37]:
model.fit(X_train, y_train, 
          validation_data=(X_val, y_val),
          batch_size=256,
          epochs=10)

Train on 16527 samples, validate on 4132 samples
Epoch 1/10
16527/16527 [==============================] - 7s 426us/sample - loss: 0.3509 - accuracy: 0.8536 - val_loss: 0.2202 - val_accuracy: 0.9061
Epoch 2/10
16527/16527 [==============================] - 4s 270us/sample - loss: 0.1939 - accuracy: 0.9187 - val_loss: 0.2091 - val_accuracy: 0.9134
Epoch 3/10
16527/16527 [==============================] - 4s 239us/sample - loss: 0.1615 - accuracy: 0.9363 - val_loss: 0.2137 - val_accuracy: 0.9068
Epoch 4/10
16527/16527 [==============================] - 4s 253us/sample - loss: 0.1410 - accuracy: 0.9460 - val_loss: 0.2316 - val_accuracy: 0.9121
Epoch 5/10
16527/16527 [==============================] - 4s 251us/sample - loss: 0.1255 - accuracy: 0.9526 - val_loss: 0.2566 - val_accuracy: 0.9000
Epoch 6/10
16527/16527 [==============================] - 4s 264us/sample - loss: 0.1125 - accuracy: 0.9582 - val_loss: 0.2890 - val_accuracy: 0.8988
Epoch 7/10
16527/16527 [===========================

Результаты у моделей похожи, немного лучше отработала GRU